## Hypothesis Tests in Python
In this assessment, you will look at data from a study on toddler sleep habits. 

The hypothesis tests you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [86]:
import numpy as np
import pandas as pd
from scipy.stats import t
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import scipy.stats.distributions as dist
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
childrenâ€™s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime and Total 24-Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data
df = pd.read_csv("nap_no_nap.csv") 

In [5]:
# First, look at the DataFrame to get a sense of the data
df.head()

id     sex  age (months)  dlmo time  days napped  napping  \
0   1  female          33.7      19.24            0        0   
1   2  female          31.5      18.27            0        0   
2   3    male          31.9      19.14            0        0   
3   4  female          31.6      19.69            0        0   
4   5  female          33.0      19.52            0        0   

   nap lights outl time  nap sleep onset  nap midsleep  nap sleep offset  \
0                   NaN              NaN           NaN               NaN   
1                   NaN              NaN           NaN               NaN   
2                   NaN              NaN           NaN               NaN   
3                   NaN              NaN           NaN               NaN   
4                   NaN              NaN           NaN               NaN   

   nap wake time  nap duration  nap time in bed  night bedtime  \
0            NaN           NaN              NaN          20.45   
1            NaN           NaN              NaN          19.23   
2            NaN           NaN              NaN          19.60   
3            NaN           NaN              NaN          19.46   
4            NaN           NaN              NaN          19.21   

   night sleep onset  sleep onset latency  night midsleep time  \
0              20.68                 0.23                 1.92   
1              19.48                 0.25                 1.09   
2              20.05                 0.45                 1.29   
3              19.50                 0.05                 1.89   
4              19.65                 0.45                 1.30   

   night wake time  night sleep duration  night time in bed  \
0             7.17                 629.4              643.0   
1             6.69                 672.4              700.4   
2             6.53                 628.8              682.6   
3             8.28                 766.6              784.0   
4             6.95                 678.0              718.0   

   24 h sleep duration  bedtime phase difference  \
0                629.4                     -1.21   
1                672.4                     -0.96   
2                628.8                     -0.46   
3                766.6                      0.23   
4                678.0                      0.31   

   sleep onset phase difference  midsleep phase difference  \
0                         -1.44                       6.68   
1                         -1.21                       6.82   
2                         -0.91                       6.15   
3                          0.19                       6.20   
4                         -0.13                       5.78   

   wake time phase difference  
0                       11.93  
1                       12.42  
2                       11.39  
3                       12.59  
4                       11.43

**Question**: What value is used in the column 'napping' to indicate a toddler takes a nap? (see reference article) 

**Questions**: What is the overall sample size $n$? What are the sample sizes of napping and non-napping toddlers?

In [11]:
print(df.columns)
df["napping"].value_counts()

Index(['id', 'sex', 'age (months)', 'dlmo time', 'days napped', 'napping',
       'nap lights outl time', 'nap sleep onset', 'nap midsleep',
       'nap sleep offset', 'nap wake time', 'nap duration', 'nap time in bed',
       'night bedtime', 'night sleep onset', 'sleep onset latency',
       'night midsleep time', 'night wake time', 'night sleep duration',
       'night time in bed', '24 h sleep duration', 'bedtime phase difference',
       'sleep onset phase difference', 'midsleep phase difference',
       'wake time phase difference'],
      dtype='object')


1    15
0     5
Name: napping, dtype: int64

## Hypothesis tests
We will look at two hypothesis test, each with $\alpha = .05$:  


1. Is the average bedtime for toddlers who nap later than the average bedtime for toddlers who don't nap?


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}>\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap}>0$$


2. The average 24 h sleep duration (in minutes) for napping toddlers is different from toddlers who don't nap.


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

First isolate `night bedtime` into two variables - one for toddlers who nap and one for toddlers who do not nap.

In [25]:
nap_bedtime = df.loc[df.napping == 1,"night bedtime"].dropna()
nap_bedtime.head()

5    19.95
6    20.60
7    22.01
8    20.24
9    20.78
Name: night bedtime, dtype: float64

In [26]:
no_nap_bedtime = df.loc[df.napping == 0, "night bedtime"].dropna()
no_nap_bedtime.head()

0    20.45
1    19.23
2    19.60
3    19.46
4    19.21
Name: night bedtime, dtype: float64

Now find the sample mean bedtime for nap and no_nap.

In [27]:
nap_mean_bedtime = nap_bedtime.mean()
nap_mean_bedtime

20.304

In [28]:
no_nap_mean_bedtime = no_nap_bedtime.mean()
no_nap_mean_bedtime

19.590000000000003

**Question**: What is the sample difference of mean bedtime for nappers minus no nappers?

In [29]:
mean_bedtime_diff = nap_mean_bedtime - no_nap_mean_bedtime
mean_bedtime_diff

0.7139999999999951

Now find the sample standard deviation for $X_{nap}$ and $X_{no\ nap}$.

In [31]:
# The np.std function can be used to find the standard deviation. The
# ddof parameter must be set to 1 to get the sample standard deviation.
# If it is not, you will be using the population standard deviation which
# is not the correct estimator
nap_s_bedtime = np.std(nap_bedtime, ddof=1)
nap_s_bedtime

0.5910619981984009

In [33]:
no_nap_s_bedtime = np.std(no_nap_bedtime, ddof=1)
no_nap_s_bedtime

0.5075923561284187

**Question**: What is the s.e.$(\bar{X}_{nap} - \bar{X}_{no\ nap})$?

We expect the variance in sleep time for toddlers who nap and toddlers who don't nap to be the same. So we use a pooled standard error.

Calculate the pooled standard error of $\bar{X}_{nap} - \bar{X}_{no\ nap}$ using the formula below.

$s.e.(\bar{X}_{nap} - \bar{X}_{no\ nap}) = \sqrt{\frac{(n_1-1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}(\frac{1}{n_1}+\frac{1}{n_2})}$

In [52]:
pooled_se = np.sqrt((14*nap_s_bedtime**2 + 4*no_nap_s_bedtime**2)*(1/15+1/5)/(20-2))
pooled_se

0.2961871280370147

**Question**: Given our sample size of $n$, how many degrees of freedom ($df$) are there for the associated $t$ distribution?

Now calculate the $t$-test statistic for our first hypothesis test using  
* pooled s.e.($\bar{X}_{nap} - \bar{X}_{no\ nap}$)  
* $\bar{X}_{nap} - \bar{X}_{no\ nap}$  
* $\mu_{0,\ nap} - \mu_{0,\ no\ nap}=0$, the population difference in means under the null hypothesis

In [62]:
tstat = mean_bedtime_diff/pooled_se
tstat

2.4106381824626966

**Question**: What is the p-value for the first hypothesis test?

For a discussion of probability density functions (PDF) and cumulative distribution functions (CDF) see:

https://integratedmlai.com/normal-distribution-an-introductory-guide-to-pdf-and-cdf/

To find the p-value, we can use the CDF for the t-distribution:
```
t.cdf(tstat, df)
```
Which for $X \sim t(df)$ returns $P(X \leq tstat)$.

Because of the symmetry of the $t$ distribution, we have that 
```
1 - t.cdf(tstat, df)
```
returns $P(X > tstat)$

The function `t.cdf(tstat, df)` will give you the same value as finding the one-tailed probability of `tstat` on a t-table with the specified degrees of freedom.

Use the function `t.cdf(tstat, df)` to find the p-value for the first hypothesis test.

In [72]:
pvalue = 1 - t.cdf(tstat, 18)
pvalue

0.013417041438843036

**Question**: What are the t-statistic and p-value for the second hypothesis test?

Calculate the $t$ test statistics and corresponding p-value using the `scipy` function `scipy.stats.ttest_ind(a, b, equal_var=True)` and check with your answer. 

**Question**: Does `scipy.stats.ttest_ind` return values for a one-sided or two-sided test?

**Question**: Can you think of a way to recover the results you got using `1-t.cdf` from the p-value given by `scipy.stats.ttest_ind`?

Use the `scipy` function `scipy.stats.ttest_ind(a, b, equal_var=True)` to find the $t$ test statistic and corresponding p-value for the second hypothesis test.

In [99]:
nap_sleep = df.loc[df.napping == 1,"24 h sleep duration"].dropna()
nap_sleep_mu = nap_sleep.mean()
nap_sleep_sd = np.std(nap_sleep,ddof=1)

no_nap_sleep = df.loc[df.napping == 0,"24 h sleep duration"].dropna()
no_nap_sleep_mu = no_nap_sleep.mean()
no_nap_sleep_sd = np.std(nap_sleep,ddof=1)

nap_sleep_diff = nap_sleep_mu - no_nap_sleep_mu

pooled_se2 = np.sqrt((14*nap_sleep_sd**2 + 4*no_nap_sleep_sd**2)*(1/15 +1/5)/(20-2))
t_stat2 = nap_sleep_diff/pooled_se2
2*dist.norm.cdf(-np.abs(t_stat2))

0.10292313526447358

In [94]:
sp.stats.ttest_ind(nap_sleep, no_nap_sleep, equal_var=True)

Ttest_indResult(statistic=1.4811248223284985, pvalue=0.1558664953018476)

**Question**: For the $\alpha=.05$, do you reject or fail to reject the first hypothesis?

**Question**: For the $\alpha=.05$, do you reject or fail to reject the second hypothesis?